# Загрузка данных


In [1]:
pip install face_recognition

     |████████████████████████████████| 100.2MB 67kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=5171317aad2b578ef17987a099f5e13d5b7e919aae3c55846d2097a2c509517b
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [2]:
import os
import pickle
import sys
import face_recognition
import cv2
from google.colab.patches import cv2_imshow
import time

from IPython.display import HTML
from base64 import b64encode

from PIL import Image, ImageDraw

In [3]:
from google.colab import drive
drive.mount('/content/drive')
directory = '/content/drive/MyDrive/Colab Notebooks/Neural_university/Соревнование/Carrey/'
os.chdir(directory)
#os.chdir('/content/')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks/Neural_university/Соревнование/Carrey'

# Тренировка на фото

Предварительно из видео вырезаны примеры изображений Керри и помещены в папку "img_carrey".
Задача - создать файл с обобщенными данными на изображение  Керри carrey_encodings.pickle


In [ ]:
# Скрипт получает изображение, делает скриншот и отправляет на сервер. Сравниват с базой и если сопадает то делает действие
# Имя человека, которого определяем
def train_model_by_img(name_acter, folder_carrey):

    if not os.path.exists(folder_carrey):
        print("[ERROR] there is no directory 'img_carrey'")
        sys.exit()

    known_encodings = []
    num_img_carrey = 0
    # Здесь несколько снимков одного человека img_carrey
    images = os.listdir(folder_carrey)
    print('Кадров для обучения:', len(images))   # Выведем имена файлов
    
    # Пробежимся по каждому изображени, загружакем в face_recognitions, снимаем кодировку и сравниваем
    # enumerate позволит кроме изображения получить его индекс - для выврода в терминал чтобы видетиь програсс
    for (i, image) in enumerate(images):     
        # print(f"[+] распознаем {i} изображение, файл - {image} из {len(images)}")
        face_img = face_recognition.load_image_file(folder_carrey + image)    #Загрузка изображдений        
        try:
          face_enc = face_recognition.face_encodings(face_img)[0]      #Кодировка каждого лица

          # Каждая полученная кодировка должна сопосталяться с полученной ранее
          # Если True - то добавляем в список
          # перовначально он пустой и добавляем туда первую кодировку
          if len(known_encodings) == 0:
              known_encodings.append(face_enc)
          
          # Если там уже естm первая кодировка, то проходим по этой кодироdке
          else:
            # face_enc - новое изображение сравниваем соо всеми в списке. Может быт долго если длинный список
              for item in range(0, len(known_encodings)):        
                  result = face_recognition.compare_faces([face_enc], known_encodings[item])
                  #print(result)      # Выводим узнал или нет Керри

                  if result[0]:
                      known_encodings.append(face_enc)     # Если True - добавляем в список и выходим из списка
                      #print("Тот же человек")
                      num_img_carrey += 1
                      break
                  else:
                      #print("Другой человек!")
                      break
        except:
         print('не узнал')
    #print(known_encodings)
    #print(f"Length {len(known_encodings)}")
    print('Узнал фоток', num_img_carrey)
    data = {
        "name": name_acter,
        "encodings": known_encodings
    }

    with open(directory + f"{name_acter}_encodings.pickle", "wb") as file:    # wb - запись в двоичном формате
        file.write(pickle.dumps(data))

    return f"[INFO] File {name_acter}_encodings.pickle successfully created"



# Распознавание человека

In [14]:
# Определим Керри на видео и сохраним эти кадры

def detect_person_in_video(name_video, name_out_video):

    # Время выполнения распознавания
    start_time = time.time()
    moment_time0 = 0

    data = pickle.loads(open('carrey_encodings.pickle', "rb").read())    # Читаем в двоичном формате
    video = cv2.VideoCapture(name_video)          # Можноб сюда данные из камеры

    num = 0
   
    # Установим формат выходного видео mp4
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # (*'XVID') (*'mp4v') ('M','J','P','G')
    # Определим частоту кадров fps в исходном видео чтобы в новом сделать такое же   
    fps = int(video.get(cv2.CAP_PROP_FPS))       # Определим число кадро в сек. в исходном видео
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))    # Определим ширину и высоту кадра
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print('fps = ' + str(fps))
    print('Разрешение кадра', width, height)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))   # Определим число кадров
    print('Число кадров исходного видео', frame_count )

    out_video = cv2.VideoWriter(name_out_video, fourcc, fps, (width, height)) # создаем новое видео
    
    for _ in range(frame_count-10):
    #while True:
        ret, image = video.read()              # Читаем и декодируем кадр
  

        # Метод face_locations библиотеки face_recognition - получаем массив координат 
        # всех человеческих лиц на каждом кадре
        # model="cnn" - для работы на gpu (по дефолту hog - это для процессоров)
        locations = face_recognition.face_locations(image, model="cnn")       # Получим массив координат лиц на кадре
        encodings = face_recognition.face_encodings(image, locations)         # получим кодировки лиц из кадров верх-право-низ-лево
 
        # Будем выводить время работы каждые 30 сек.        
        moment_time = time.time() - start_time - moment_time0
        if (29.85 < moment_time < 30):
          print('Работаем уже:', round(time.time() - start_time, 3), ' сек')
          moment_time0 = time.time() - start_time

        # В цикле проходим по колдировкам и локациям
        # result - Сравнение кодировки лиц из датасета data["encodings"] с кодировками лиц face_encoding из каждого кадра.
        # Если совпадает - делаем действие (выделяем лицо на видео в оригинальном примере)
        # Циклом проходим по кодировкам и локациям
        # Первое - из датасета, вторая из кадра
        for face_encoding, face_location in zip(encodings, locations):      
            result = face_recognition.compare_faces(data["encodings"], face_encoding)    

            match = None                        # Совпадение с оригиналом
            if True in result:                  # result возвращает True eсли совпадает 
                out_video.write(image)          # Сохраняем в файл видео кадр  - он как раз с Джимом Керри
                num += 1                        # Считаем число кадров с Керри

    # Посчитаем время фильма, num - число кадров. 
    # Взять данные из видео нельзя, т.к. фильм сохраняется на диске около 2 мин. и программа его не видит сразу
    duration = round(num/fps, 3)

    print('Продолжительность созданного видео, сек ' + str(duration))
    minutes = int(duration/60)
    seconds = duration%60
    print('Продолжительность созданного видео = ' + str(minutes) + ' мин' + ':' + str(seconds) + ' сек')

    # конвертируем в mp4, как просят по заданию
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = name_out_video, output = name_out_video))


    out_video.release()
    video.release()
    cv2.destroyAllWindows()

#detect_person_in_video('Shou_Trumana_Kinosimka3.mp4', 'frame_carrey4')



# Главная программа

In [15]:
def main():
    # Файл видео для распознавания
    name_video = "Shou_Trumana_Kinosimka.RU (online-video-cutter.com).mp4"    # Имя файла исходного видео
    #name_video = 'Shou_Trumana_Kinosimka1.mp4'      # Имя файла исходного видео - rjhjnrjt lkz nhtybhjdrb
    name_out_video = 'out_video_carry_full_1-1.avi'    # Имя файлы получившегося видео с актером
    foldet_scrinshots = 'frame_carrey/'  # Папка для хранения всех найденных кадров с актером frame_carrey3 - для тренировочного видео
    folder_carrey = 'img_carrey/'     # Папка для хранения образцов лица актера
    name_acter = 'carrey'

    print('Тренируем скрипт..')
    #train_model_by_img(name_acter, folder_carrey)    # Если файл carrey_encodings.pickle уже есть, то тренировать заново не надо

    print('Распознаем Керри..')
    detect_person_in_video(name_video, name_out_video)

    print('Приятного просмотра!')


if __name__ == '__main__':
    main()


Тренируем скрипт..
Распознаем Керри..
fps = 23
Разрешение кадра 720 408
Число кадров исходного видео 29629
Работаем уже: 29.898  сек
Работаем уже: 59.761  сек
Работаем уже: 89.612  сек
Работаем уже: 119.478  сек
Работаем уже: 149.378  сек
Работаем уже: 179.241  сек
Работаем уже: 209.115  сек
Работаем уже: 239.014  сек
Работаем уже: 268.892  сек
Работаем уже: 298.752  сек
Работаем уже: 328.63  сек
Работаем уже: 358.508  сек
Работаем уже: 388.373  сек
Работаем уже: 418.244  сек
Работаем уже: 448.154  сек
Работаем уже: 478.042  сек
Работаем уже: 507.925  сек
Работаем уже: 537.823  сек
Работаем уже: 567.697  сек
Работаем уже: 597.567  сек
Работаем уже: 627.469  сек
Работаем уже: 657.343  сек
Работаем уже: 687.226  сек
Работаем уже: 717.121  сек
Работаем уже: 747.003  сек
Работаем уже: 776.895  сек
Работаем уже: 806.79  сек
Работаем уже: 836.688  сек
Работаем уже: 866.574  сек
Работаем уже: 896.477  сек
Работаем уже: 926.369  сек
Работаем уже: 956.233  сек
Работаем уже: 986.098  сек
Работае